In [1]:
%%capture
import pandas as pd
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from mlflow.utils.environment import _mlflow_conda_env
import shutil
from glob import glob
import os

iris = datasets.load_iris()
iris_train = pd.DataFrame(iris.data, columns=iris.feature_names)
clf = RandomForestClassifier(max_depth=7, random_state=0)
clf.fit(iris_train, iris.target)

# Infer the signature from the training dataset and model's predictions
signature = infer_signature(iris_train, clf.predict(iris_train))

# Conda environment
custom_env = _mlflow_conda_env(
    additional_conda_deps=None,
    additional_pip_deps=["xgboost==1.5.2"],
    additional_conda_channels=None,
)

# Sample
input_sample = iris_train.sample(n=1)
output_sample = clf.predict(input_sample)

# Delete previous experiments by erase the folser mlruns
if os.path.isdir("mlruns"):
    shutil.rmtree("./mlruns")

# Log the scikit-learn model with the custom signature
mlflow.sklearn.log_model(
    clf,
    artifact_path="iris_rf",
    conda_env=custom_env,
    signature=signature,
    input_example=input_sample,
)

In [ ]:
%%writefile models/score.py
import logging
import os
import json
import mlflow
from io import StringIO
from mlflow.pyfunc.scoring_server import infer_and_parse_json_input, predictions_to_json
import joblib


def init():
    global model
    global input_schema
    # "model" is the path of the mlflow artifacts when the model was registered. For automl
    # models, this is generally "mlflow-model".
    model_path = os.path.join(os.getenv("AZUREML_MODEL_DIR"), "model.pkl")
    model = joblib.load(model_path)
    input_schema = model.metadata.get_input_schema()
    logging.info("Init complete")


def run(raw_data):
    json_data = json.loads(raw_data)
    if "input_data" not in json_data.keys():
        raise Exception("Request must contain a top level key named 'input_data'")

    serving_input = json.dumps(json_data["input_data"])
    data = infer_and_parse_json_input(serving_input, input_schema)
    predictions = model.predict(data)

    result = StringIO()
    predictions_to_json(predictions, result)
    logging.info("Request processed")
    return result.getvalue()

Overwriting models/score.py
